In [1]:
import re
import pandas as pd
from pandas import DataFrame as df
import numpy as np
import emoji

In [2]:
twitter_file = open("data/raw_korean.txt", "r", encoding = "utf-8")
twitter = twitter_file.read()
twitter_file.close()

twitter[:500]

'RT @haha247979: @onemoreonce4bts @BTS_twt 넵넵💜 저..  늦게 해시 투표 동참한거 사실.. 조금.. 많이 죄송하긴 해써요😂😂\n열심히 슨배님덜 따라갈게요~😍\n\nButter by BTS #BTS\nStream #BTS_B…@green_zzang 맞아맞아 그렇긴 하지! 😂 근데 난 내용이 맘에 안 들거나 더 안 읽어도 되겠다 싶으면 다 안 읽었는데도 읽은 책처럼 생각해(!) ㅋㅋㅋㅋㅋ@weareDRIPPIN 오늘의 한마디를 주세요!😂\n\n#DRIPPIN #드리핀 #Villain #드리핀과_함께하는_멘션파티@KnitOhlala 이거 때문에 어제 실 주문을 한지라 ㅋㅋㅋㅋ 매우 찔립니다요!!! 근데 넘 잘 입을 것 같아서 함뜨참가! 얍! 색조합 잘 못하는데 원작하고 비슷하게 갑니다 😂@shotasaigo 까까님😂😂 생일메세지 정말 감사해요 🥺🥺💙 끝까지 덕질해라는 뜻인가봐요😇 까까님도 남은 시간 즐겁게 보내세요 🥰🥰아 내스급 성현제 책갈피가 아니라 옷핀입니다😂😂'

In [3]:
#0206 data added
twitter_file_0206 = open("data/raw_korean_0206.txt", "r", encoding = "utf-8")
twitter += twitter_file_0206.read()
twitter_file_0206.close()

twitter[:500]

'RT @haha247979: @onemoreonce4bts @BTS_twt 넵넵💜 저..  늦게 해시 투표 동참한거 사실.. 조금.. 많이 죄송하긴 해써요😂😂\n열심히 슨배님덜 따라갈게요~😍\n\nButter by BTS #BTS\nStream #BTS_B…@green_zzang 맞아맞아 그렇긴 하지! 😂 근데 난 내용이 맘에 안 들거나 더 안 읽어도 되겠다 싶으면 다 안 읽었는데도 읽은 책처럼 생각해(!) ㅋㅋㅋㅋㅋ@weareDRIPPIN 오늘의 한마디를 주세요!😂\n\n#DRIPPIN #드리핀 #Villain #드리핀과_함께하는_멘션파티@KnitOhlala 이거 때문에 어제 실 주문을 한지라 ㅋㅋㅋㅋ 매우 찔립니다요!!! 근데 넘 잘 입을 것 같아서 함뜨참가! 얍! 색조합 잘 못하는데 원작하고 비슷하게 갑니다 😂@shotasaigo 까까님😂😂 생일메세지 정말 감사해요 🥺🥺💙 끝까지 덕질해라는 뜻인가봐요😇 까까님도 남은 시간 즐겁게 보내세요 🥰🥰아 내스급 성현제 책갈피가 아니라 옷핀입니다😂😂'

In [4]:
kaggle_file = open("data/kaggle_translated.txt", "r", encoding = "utf-8")
kaggle = kaggle_file.read()
kaggle = kaggle.replace("⁇", "")
kaggle_file.close()

kaggle[:500]

'2차전에 도착하는 스쿼드   🚀 남자는 5x8 140 파운드와 같다 그의 거시기는 길고 강했다 (항상 작은 녀석은         )   🙃 "팔로워스"👇 나는 호흡을 한다.💀 우리의 스케줄이 떨어질 때까지 2 {4} 시간!! "새"n 나는 밀크 보이 스튜디오에서 살고있다 - 히트 곡을 만드는 집   🏾 새들이 너무 무서워요  🤧 그게 나야.   😂 마음이 너무 충만해요 rn   💖 이 에피소드에서 내가 가장 좋아하는 노래 중 하나입니다   ️ 염소를 데리고 처음으로 몸치장을 했어요   😂 "새") 이번 여름에 나는 최고의 삶을 살고있다 &amp; 나는 그것을 의미한다.💯    알려줘요   🏾 이번 여름에 나는 최고의 삶을 살고있다 &amp; 나는 그것을 의미한다.💯    [MTV LATIN AMERICA AWARDS] | 1:00 - 2:00 AM KST | KPOP REVOLUTION 1. : 130.7k    2.… 군대, 누가 리트윗하는지 따라와요   🔥 퍼펙트 다 앨리 브룩 '

In [5]:
text = twitter + kaggle
# text = twitter

In [6]:
#영문, 특수문자(? ! . 제외), 숫자 제거
text = re.sub('[a-zA-z]','',text)
text = re.sub('[\{\}\[\]\/;:|\)*~`^\-_+<>@\#$%&\\\=\(\'\"\♥\♡\n0-9]','',text)
text = text.strip()
text[-500:]

'좋은 밤의 일   ️ 이온은 욜이 그를 쓰레기라고 불렀을 때 그의 거짓말을 전혀 보고 싶지 않다.😠 둘 다 정확합니다.😜 방탄 쉐어링  직원을위한 음식을 사는 것은 나를 부드럽게 만든다, 그들은 너무 달콤하다   💔 진짜 ᅲᅲ🔥    모두가 말하는 것을들을 수 있습니까?   🇸 때때로 나는 내가 어떻게 이런 박자를 생각해냈는지 궁금하다. 신은 칭찬을 받을 자격이 있다.💜 이 귀염둥이를보십시오    많은 기다리고있는 영화를 위해 함께 돌아올 것입니다. 나는 이 노래를 정말 좋아한다.😂 우정을 고치고 신뢰를 회복하는 것이 서로에게 노래를 부르는 것만 큼 쉬웠다면   😍 이 땅에서 년을 살았다.😍 저한텐 매일매일이에요   🖤 저한텐 매일매일이에요   🖤   엿 먹어라, 나는 나의 델타를 보려고 한다.😍 너무 잘 알고 있어요  😂 무대 위의 명의 왕자들😍 나는 백금 금발 머리를 원하지만 나는 건강한 머리카락을 너무 좋아한다.😢 와인이 정말 마시고 싶어요   😩 새 스냅챗 업데이트!!   🏼'

In [7]:
emojis = emoji.emoji_lis(text)
emojis[:10]

[{'location': 2, 'emoji': '💜'},
 {'location': 44, 'emoji': '😂'},
 {'location': 45, 'emoji': '😂'},
 {'location': 60, 'emoji': '😍'},
 {'location': 80, 'emoji': '😂'},
 {'location': 157, 'emoji': '😂'},
 {'location': 264, 'emoji': '😂'},
 {'location': 269, 'emoji': '😂'},
 {'location': 270, 'emoji': '😂'},
 {'location': 286, 'emoji': '🥺'}]

In [8]:
emojis = list(set([i['emoji'] for i in emojis]))
print(len(emojis))
emojis[:10]

522


['😮', '🌰', '👴', '😋', '🎂', '🐦', '🙃', '👼🏻', '🆙', '👦']

In [9]:
delimiters = ['.', '!', '?', 'ㅎ', 'ㅋ', ' ']
delimiters.extend(emojis)

#두번이상 반복된 것 한 번으로 축소
for d in delimiters:
    flag = False
    while True:
        if text.find(d+d) == -1:
            break
        text = text.replace(d+d, d)

text[:500]

'넵넵💜 저. 늦게 해시 투표 동참한거 사실. 조금. 많이 죄송하긴 해써요😂열심히 슨배님덜 따라갈게요😍 … 맞아맞아 그렇긴 하지! 😂 근데 난 내용이 맘에 안 들거나 더 안 읽어도 되겠다 싶으면 다 안 읽었는데도 읽은 책처럼 생각해! ㅋ 오늘의 한마디를 주세요!😂 드리핀 드리핀과함께하는멘션파티 이거 때문에 어제 실 주문을 한지라 ㅋ 매우 찔립니다요! 근데 넘 잘 입을 것 같아서 함뜨참가! 얍! 색조합 잘 못하는데 원작하고 비슷하게 갑니다 😂 까까님😂 생일메세지 정말 감사해요 🥺💙 끝까지 덕질해라는 뜻인가봐요😇 까까님도 남은 시간 즐겁게 보내세요 🥰아 내스급 성현제 책갈피가 아니라 옷핀입니다😂 뮤지광컴퍼니는 끝났지만 나는 멈추지 않는다.정규직 기다려라.😂모두들 감사합니다❤️👍유튜브는 아직 남았어용ㅎ크래비티 형준 아이돌라이브 뮤지광 . 우왕😍 솜사탕😙차배치가 초컷이래서😂 우리껀 춘절이후에 받아욥요기는 솜사탕 차폼이용😁 . 에에 아직 못갈겼어 담달에 살구야!😂🔥 흐뭇하게 시작해서 중간에 탈'

In [10]:
df = df(columns = ['x', 'y'])
df

,x,y


In [11]:
delimiters.remove(' ')
x = ""

for i in range(len(text)):
    if text[i] in delimiters:
#         if i+1 < len(text) and text[i+1] in delimiters:
#             continue
        if len(x) < 10:
            continue
        if text[i] not in emojis:
            df = df.append({'x': x, 'y': 'N'}, ignore_index = True)
        else:
            df = df.append({'x': x, 'y': text[i]}, ignore_index = True)
        x = ""
    else:
        x += text[i]

In [12]:
len(df)

6645

In [13]:
#중복 제거
df = df.drop_duplicates(subset = ['x']).reset_index(drop = True)

In [14]:
print(len(df))
df.head() #2231 -> 5725 

4385


,x,y
0,넵넵 저 늦게 해시 투표 동참한거 사실,N
1,조금 많이 죄송하긴 해써요,😂
2,열심히 슨배님덜 따라갈게요,😍
3,… 맞아맞아 그렇긴 하지,N
4,근데 난 내용이 맘에 안 들거나 더 안 읽어도 되겠다 싶으면 다 안 읽었는데도 ...,N


In [15]:
#x; 한국어만 남기기
df['x'] = [re.compile('[^ ㄱ-ㅣ가-힣]+').sub('', df['x'][i]).strip() for i in range(len(df['x']))]

In [16]:
len(df) #total data length

4385

In [17]:
df_count = df['y'].value_counts(sort = True)

len(df_count) #unique labels

321

In [18]:
#count 10 이하인 것 제거
# more_than_ten = df_count[df_count > 10]

#라벨 수를 최빈도 30개로
frequent_labels = list(df_count.head(30).index)
df = df[df['y'].isin(frequent_labels)]
df.head()

,x,y
0,넵넵 저 늦게 해시 투표 동참한거 사실,N
1,조금 많이 죄송하긴 해써요,😂
2,열심히 슨배님덜 따라갈게요,😍
3,맞아맞아 그렇긴 하지,N
4,근데 난 내용이 맘에 안 들거나 더 안 읽어도 되겠다 싶으면 다 안 읽었는데도 읽은...,N


In [19]:
len(df) #total data length

3076

In [20]:
len(df_count) #unique labels

321

In [21]:
df_count = df['y'].value_counts(sort = True)
print(len(df_count))
df_count

30


N    2346
😂      80
❤      53
😭      49
😍      32
💕      28
💜      26
🔥      25
👍      24
🥺      24
💖      24
❣      23
🙏      23
😆      23
✨      23
😊      22
😘      21
🥰      20
😎      19
💗      19
💙      19
🤭      18
💚      18
👉      17
✌      17
😁      17
💯      17
😑      17
🤤      16
🐹      16
Name: y, dtype: int64

In [22]:
#undersample data with label 'N'
tail_len = int(len(df[df['y'] == 'N']) * 0.9)
to_drop = df[df['y']=='N'].tail(tail_len)
df = df.drop(to_drop.index)
df.reset_index(drop = True)

,x,y
0,넵넵 저 늦게 해시 투표 동참한거 사실,N
1,조금 많이 죄송하긴 해써요,😂
2,열심히 슨배님덜 따라갈게요,😍
3,맞아맞아 그렇긴 하지,N
4,근데 난 내용이 맘에 안 들거나 더 안 읽어도 되겠다 싶으면 다 안 읽었는데도 읽은...,N
...,...,...
960,내 이마에 대고 말할 수 있는 여자는 한 명뿐이고 우리 엄마도 아니에요,😂
961,진짜 역대급 귀여워요 옴,😍
962,평화와 사랑을 그대에게,❤
963,우정을 고치고 신뢰를 회복하는 것이 서로에게 노래를 부르는 것만 큼 쉬웠다면,😍


In [23]:
df_count = df['y'].value_counts(sort = True)
print(len(df_count))
df_count

30


N    235
😂     80
❤     53
😭     49
😍     32
💕     28
💜     26
🔥     25
👍     24
🥺     24
💖     24
❣     23
🙏     23
😆     23
✨     23
😊     22
😘     21
🥰     20
😎     19
💗     19
💙     19
🤭     18
💚     18
👉     17
✌     17
😁     17
💯     17
😑     17
🤤     16
🐹     16
Name: y, dtype: int64

In [24]:
print(len(df))
df.tail(10)

965


,x,y
4332,세훈이 촬영 때문에 바빠요 비비랑 아직도 같이 있으면 좋겠어요,😘
4340,가 다시 한번 그것을한다면,💯
4351,파산도 드래곤볼츠,😍
4356,파산도 드래곤볼츠,😍
4358,진짜 역대급 귀여워요 옴,😍
4359,내 이마에 대고 말할 수 있는 여자는 한 명뿐이고 우리 엄마도 아니에요,😂
4360,진짜 역대급 귀여워요 옴,😍
4361,평화와 사랑을 그대에게,❤
4374,우정을 고치고 신뢰를 회복하는 것이 서로에게 노래를 부르는 것만 큼 쉬웠다면,😍
4378,너무 잘 알고 있어요,😂


In [25]:
df.to_csv('data/twitter_clean.csv', index=False)